# Tutorial 4: Creating and Managing Optical Interfaces

## Learning Objectives
- Create interfaces on mounted devices via TransportPCE
- Use TransportPCE device renderer services
- Verify interface creation and status
- Prepare devices for network connections

## Prerequisites
- Completed Tutorials 1-3
- Devices successfully mounted to TransportPCE
- Understanding of OpenROADM device models


## Step 1: Student Configuration

In [ ]:
# === STUDENT CONFIGURATION ===
STUDENT_ID = "1"  # Change this to your assigned student ID

# Generated configuration
NETWORK_NAME = f"student{STUDENT_ID}-network"
TPCE_CONTAINER_NAME = f"student{STUDENT_ID}-tpce"

print(f"Student ID: {STUDENT_ID}")
print(f"Network: {NETWORK_NAME}")
print(f"TransportPCE Container: {TPCE_CONTAINER_NAME}")

## Step 2: Import Libraries and Setup

In [ ]:
import docker
import requests
import json
import time
from requests.auth import HTTPBasicAuth
from datetime import datetime
from common_utils import StatusIndicators, print_status, print_section, DockerHelper

# Initialize Docker client
docker_helper = DockerHelper()

# Disable SSL warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

print_status("Libraries imported successfully", StatusIndicators.SUCCESS)

## Step 3: Interface Management Functions

In [ ]:
# Hardcoded service parameters based on production robot tests
SERVICE_PARAMS = {
    "modulation-format": "dp-qam16",
    "center-freq": 196.08125,
    "nmc-width": 75,
    "mc-width": 87.5,
    "min-freq": 196.0375,
    "max-freq": 196.125,
    "lower-spectral-slot-number": 755,
    "higher-spectral-slot-number": 768
}

class InterfaceManager:
    """Manages interface operations through TransportPCE device renderer"""
    
    def __init__(self, tpce_ip):
        self.tpce_ip = tpce_ip
        self.base_url = f"http://{tpce_ip}:8181"
        self.auth = HTTPBasicAuth('admin', 'admin')
        self.session = requests.Session()
        self.session.auth = self.auth
        self.headers = {
            'Accept': 'application/json',
            'Content-Type': 'application/json'
        }
    
    def get_mounted_devices(self):
        """Get list of mounted devices"""
        endpoint = "/rests/data/network-topology:network-topology/topology=topology-netconf"
        response = self.session.get(f"{self.base_url}{endpoint}", headers=self.headers)
        
        if response.status_code == 200:
            data = response.json()
            topology = data.get('network-topology:topology', [])
            nodes = topology[0].get('node', []) if topology else []
            return [{
                'node_id': node.get('node-id'),
                'status': node.get('netconf-node-topology:netconf-node', {}).get('connection-status'),
                'type': 'ROADM' if 'roadm' in node.get('node-id', '').lower() else 'TRANSPONDER'
            } for node in nodes if node.get('netconf-node-topology:netconf-node', {}).get('connection-status') == 'connected']
        return []
    
    def get_device_interfaces(self, device_id):
        """Get interfaces from a specific device"""
        endpoint = f"/rests/data/network-topology:network-topology/topology=topology-netconf/node={device_id}/yang-ext:mount/org-openroadm-device:org-openroadm-device?content=nonconfig&fields=interface"
        response = self.session.get(f"{self.base_url}{endpoint}", headers=self.headers)
        
        if response.status_code == 200:
            try:
                data = response.json()
                device_data = data.get('org-openroadm-device:org-openroadm-device', {})
                interfaces = device_data.get('interface', [])
                return interfaces
            except:
                pass
        return []
    
    def create_roadm_interfaces(self, device_id):
        """Create ROADM interfaces using TransportPCE device renderer"""
        
        print(f"\nCreating ROADM interfaces on {device_id}...")
        
        # Step 1: Create DEG1-TTP to SRG1-PP1 path
        payload1 = {
            **SERVICE_PARAMS,
            "operation": "create",
            "service-name": f"roadm-service-1-deg-srg",
            "wave-number": "0",
            "nodes": [{
                "node-id": device_id,
                "src-tp": "DEG1-TTP-TXRX",
                "dest-tp": "SRG1-PP1-TXRX"
            }]
        }
        
        endpoint = "/rests/operations/transportpce-device-renderer:service-path"
        full_url = f"{self.base_url}{endpoint}"

        # Print request details
        print("\n" + "="*80)
        print("REQUEST 1: DEG1-TTP to SRG1-PP1")
        print("="*80)
        print(f"URL: {full_url}")
        print(f"Method: POST")
        print(f"Headers: {self.headers}")
        print(f"\nPayload:")
        print(json.dumps({"input": payload1}, indent=2))
        print("="*80)

        response1 = self.session.post(full_url,
                                     headers=self.headers,
                                     json={"input": payload1})

        print(f"\nDEG1 to SRG1 Response: {response1.status_code}")
        print(f"Response: {response1.text}")

        # Step 2: Create SRG1-PP1 to DEG1-TTP path (bidirectional)
        payload2 = {
            **SERVICE_PARAMS,
            "operation": "create",
            "service-name": f"roadm-service-2-srg-deg",
            "wave-number": "0",
            "nodes": [{
                "node-id": device_id,
                "src-tp": "SRG1-PP1-TXRX",
                "dest-tp": "DEG1-TTP-TXRX"
            }]
        }

        # Print request details
        print("\n" + "="*80)
        print("REQUEST 2: SRG1-PP1 to DEG1-TTP")
        print("="*80)
        print(f"URL: {full_url}")
        print(f"Method: POST")
        print(f"Headers: {self.headers}")
        print(f"\nPayload:")
        print(json.dumps({"input": payload2}, indent=2))
        print("="*80)

        response2 = self.session.post(full_url,
                                     headers=self.headers,
                                     json={"input": payload2})

        print(f"\nSRG1 to DEG1 Response: {response2.status_code}")
        print(f"Response: {response2.text}")

        success = response1.status_code == 200 and response2.status_code == 200
        return success

    def create_transponder_interfaces(self, device_id):
        """Create transponder interfaces using TransportPCE device renderer"""

        print(f"\nCreating transponder interfaces on {device_id}...")

        # Create client to network path
        payload = {
            **SERVICE_PARAMS,
            "operation": "create",
            "service-name": f"tpdr-service",
            "wave-number": "0",
            "nodes": [{
                "node-id": device_id,
                "src-tp": "XPDR1-CLIENT1",
                "dest-tp": "XPDR1-NETWORK1"
            }]
        }

        endpoint = "/rests/operations/transportpce-device-renderer:service-path"
        full_url = f"{self.base_url}{endpoint}"

        # Print request details
        print("\n" + "="*80)
        print("REQUEST: XPDR1-CLIENT1 to XPDR1-NETWORK1")
        print("="*80)
        print(f"URL: {full_url}")
        print(f"Method: POST")
        print(f"Headers: {self.headers}")
        print(f"\nPayload:")
        print(json.dumps({"input": payload}, indent=2))
        print("="*80)

        response = self.session.post(full_url,
                                   headers=self.headers,
                                   json={"input": payload})

        print(f"\nResponse Status: {response.status_code}")
        print(f"Response: {response.text}")
        
        return response.status_code == 200

print("Interface management functions defined")
print("\nService Parameters:")
for key, value in SERVICE_PARAMS.items():
    print(f"  {key}: {value}")

### Service Parameter Cheatsheet (C-band, ITU-T G.694.1 flex-grid)

- **`modulation-format: dp-qam16`** — Dual-polarization 16-QAM.
- **`center-freq: 196.08125`** — Optical carrier center frequency (**THz**). 196.08125 THz ≈ 1528.9 nm → in the **C-band**.
- **`nmc-width: 75`** — **Network Media Channel** width (**GHz**) for planning (excludes dead-band).
- **`mc-width: 87.5`** — Actual occupied **Media Channel** width (**GHz**) = nmc + dead-band.
- **`min-freq: 196.0375` / `max-freq: 196.125`** — Lower/upper frequency edges (**THz**); span ≈ 87.5 GHz around the center.
- **`lower-spectral-slot-number: 755` / `higher-spectral-slot-number: 768`** — Flex-grid slot indices (6.25 GHz granularity).
  Inclusive width: \((768 − 755 + 1) × 6.25 = 87.5\) GHz ⇒ matches `mc-width`.


## Step 4: Initialize and Discover Devices

In [ ]:
# Get TransportPCE IP and discover devices
tpce_ip = docker_helper.get_container_ip(TPCE_CONTAINER_NAME, NETWORK_NAME)

if tpce_ip:
    interface_manager = InterfaceManager(tpce_ip)
    
    # Discover LightyNode devices in student network
    try:
        network = docker_helper.client.networks.get(NETWORK_NAME)
        network_info = docker_helper.client.api.inspect_network(network.id)
        containers = network_info.get('Containers', {})
        
        devices = []
        for container_info in containers.values():
            name = container_info.get('Name', '')
            
            if 'roadm' in name.lower() or 'tpdr' in name.lower():
                device_type = 'ROADM' if 'roadm' in name.lower() else 'TRANSPONDER'
                # Extract device name (roadm-a or tpdr-a)
                device_id = name.replace(f'student{STUDENT_ID}-', '')
                
                devices.append({
                    'node_id': device_id,
                    'type': device_type,
                    'status': 'connected'
                })
        
        print(f"Devices in student {STUDENT_ID} network: {len(devices)}")
        for device in devices:
            print(f"  - {device['node_id']} ({device['type']})")
            
    except Exception as e:
        print(f"Error discovering devices: {e}")
        devices = []
        
    if not devices:
        print("WARNING: No devices found. Please complete Tutorial 1-3 first.")
        interface_manager = None
else:
    print("ERROR: Could not connect to TransportPCE")
    interface_manager = None
    devices = []

## Step 5: Examine Existing Interfaces

In [ ]:
if interface_manager and devices:
    print("=== EXAMINING EXISTING INTERFACES ===")
    
    for device in devices:  # Check first 2 devices
        device_id = device['node_id']
        device_type = device['type']
        print(f"\n--- Interfaces on {device_id} ({device_type}) ---")
        
        interfaces = interface_manager.get_device_interfaces(device_id)
        
        if interfaces:
            print(f"Total interfaces found: {len(interfaces)}")
            
            # Show first few interface names
            for i, iface in enumerate(interfaces[:50]):
                name = iface.get('name', 'Unknown')
                iface_type = iface.get('type', 'unknown').split(':')[-1] if ':' in iface.get('type', '') else iface.get('type', 'unknown')
                admin_state = iface.get('administrative-state', 'unknown')
                oper_state = iface.get('operational-state', 'unknown')
                print(f"  {i+1}. {name} ({iface_type})")
                print(f"     Admin: {admin_state}, Oper: {oper_state}")

        else:
            print("No interfaces found or unable to retrieve")

else:
    print("Cannot examine interfaces - no devices available")

# Demonstrate interface creation with ROADM device using device renderer
if interface_manager and devices:
    print("=== DEMONSTRATION: CREATING ROADM INTERFACES VIA DEVICE RENDERER ===")
    
    # Find a ROADM device for demonstration
    roadm_devices = [d for d in devices if d['type'] == 'ROADM']
    
    if roadm_devices:
        demo_device = roadm_devices[0]
        device_id = demo_device['node_id']
        
        print(f"\nDevice: {device_id}")
        
        # Get interfaces BEFORE creation
        interfaces_before = interface_manager.get_device_interfaces(device_id)
        interface_names_before = [i.get('name', '') for i in interfaces_before]
        print(f"Interfaces before: {len(interfaces_before)}")
        
        print(f"\n--- Creating ROADM Interfaces ---")
        
        # Show the request details
        endpoint = "/rests/operations/transportpce-device-renderer:service-path"
        url = f"{interface_manager.base_url}{endpoint}"
        
        # Payload for DEG1 to SRG1
        payload1 = {
            **SERVICE_PARAMS,
            "operation": "create",
            "service-name": f"roadm-service-1-deg-srg",
            "wave-number": "0",
            "nodes": [{
                "node-id": device_id,
                "src-tp": "DEG1-TTP-TXRX",
                "dest-tp": "SRG1-PP1-TXRX"
            }]
        }
        
        print(f"Request 1 - DEG1 to SRG1:")
        print(f"  URL: {url}")
        print(f"  Method: POST")
        print(f"  Payload: {json.dumps(payload1, indent=2)}")
        
        # Make first request
        response1 = interface_manager.session.post(url, 
                                                   headers=interface_manager.headers,
                                                   json={"input": payload1})
        print(f"\nResponse 1: {response1.status_code}")
        print(f"Response: {response1.text}")
        
        # Payload for SRG1 to DEG1 (bidirectional)
        payload2 = {
            **SERVICE_PARAMS,
            "operation": "create",
            "service-name": f"roadm-service-2-srg-deg",
            "wave-number": "0",
            "nodes": [{
                "node-id": device_id,
                "src-tp": "SRG1-PP1-TXRX",
                "dest-tp": "DEG1-TTP-TXRX"
            }]
        }
        
        print(f"\nRequest 2 - SRG1 to DEG1:")
        print(f"  URL: {url}")
        print(f"  Method: POST")
        print(f"  Payload: {json.dumps(payload2, indent=2)}")
        
        # Make second request
        response2 = interface_manager.session.post(url,
                                                   headers=interface_manager.headers,
                                                   json={"input": payload2})
        print(f"\nResponse 2: {response2.status_code}")
        print(f"Response: {response2.text}")
        
        if response1.status_code == 200 and response2.status_code == 200:
            print(f"\n✓ Both API calls successful!")
            
            # Wait for interfaces to appear
            print(f"Waiting 5 seconds...")
            time.sleep(5)
            
            # Get interfaces AFTER creation
            interfaces_after = interface_manager.get_device_interfaces(device_id)
            interface_names_after = [i.get('name', '') for i in interfaces_after]
            print(f"\nInterfaces after: {len(interfaces_after)}")
            
            # Show what's new
            new_interfaces = [name for name in interface_names_after if name not in interface_names_before]
            if new_interfaces:
                print(f"New interfaces created: {len(new_interfaces)}")
                for name in new_interfaces:
                    print(f"  - {name}")
            else:
                print("No new interfaces detected (may still be initializing)")
        else:
            print(f"\n✗ API calls failed")
    else:
        print("No ROADM devices available")
        
        # Try with transponder if no ROADM
        tpdr_devices = [d for d in devices if d['type'] == 'TRANSPONDER']
        if tpdr_devices:
            print("\nUsing transponder device...")
            device_id = tpdr_devices[0]['node_id']
            
            # Get interfaces BEFORE
            interfaces_before = interface_manager.get_device_interfaces(device_id)
            print(f"Interfaces before: {len(interfaces_before)}")
            
            print(f"\n--- Creating Transponder Interfaces ---")
            
            # Show request details
            endpoint = "/rests/operations/transportpce-device-renderer:service-path"
            url = f"{interface_manager.base_url}{endpoint}"
            
            payload = {
                **SERVICE_PARAMS,
                "operation": "create",
                "service-name": f"tpdr-service",
                "wave-number": "0",
                "nodes": [{
                    "node-id": device_id,
                    "src-tp": "XPDR1-CLIENT1",
                    "dest-tp": "XPDR1-NETWORK1"
                }]
            }
            
            print(f"Request:")
            print(f"  URL: {url}")
            print(f"  Method: POST")
            print(f"  Payload: {json.dumps(payload, indent=2)}")
            
            # Make request
            response = interface_manager.session.post(url,
                                                     headers=interface_manager.headers,
                                                     json={"input": payload})
            print(f"\nResponse: {response.status_code}")
            print(f"Response: {response.text}")
            
            if response.status_code == 200:
                print(f"\n✓ API call successful!")
                time.sleep(5)
                
                # Get interfaces AFTER
                interfaces_after = interface_manager.get_device_interfaces(device_id)
                print(f"Interfaces after: {len(interfaces_after)}")
                
                # Show new interfaces
                new_count = len(interfaces_after) - len(interfaces_before)
                if new_count > 0:
                    print(f"Created {new_count} new interfaces")
            else:
                print(f"\n✗ API call failed")
else:
    print("Cannot demonstrate - missing prerequisites")

In [ ]:
if interface_manager and devices:
    print("=== DEMONSTRATION: CREATING ROADM INTERFACES VIA DEVICE RENDERER ===")

    # Find a ROADM device for demonstration
    roadm_devices = [d for d in devices if d['type'] == 'ROADM']

    if roadm_devices:
        demo_device = roadm_devices[0]
        device_id = demo_device['node_id']

        print(f"\nDevice: {device_id}")

        # Get interfaces BEFORE creation
        interfaces_before = interface_manager.get_device_interfaces(device_id)
        interface_names_before = [i.get('name', '') for i in interfaces_before]
        print(f"Interfaces before: {len(interfaces_before)}")

        # Create interfaces
        print(f"\nCreating ROADM interfaces...")
        success = interface_manager.create_roadm_interfaces(device_id)

        if success:
            print(f"\n✓ API call successful!")

            # Wait for interfaces to appear
            print(f"Waiting 5 seconds...")
            time.sleep(5)

            # Get interfaces AFTER creation
            interfaces_after = interface_manager.get_device_interfaces(device_id)
            interface_names_after = [i.get('name', '') for i in interfaces_after]
            print(f"\nInterfaces after: {len(interfaces_after)}")

            # Show what's new
            new_interfaces = [name for name in interface_names_after if name not in interface_names_before]
            if new_interfaces:
                print(f"New interfaces created: {len(new_interfaces)}")
                for name in new_interfaces:
                    print(f"  - {name}")
            else:
                print("No new interfaces detected (may still be initializing)")
        else:
            print(f"\n✗ API call failed")
    else:
        print("No ROADM devices available. Please mount a ROADM device first.")
else:
    print("Cannot demonstrate - missing prerequisites")

## Step 7: Examine Interfaces After Configuration

Now let's check the interfaces again to see what was created:

In [ ]:
if interface_manager and devices:
    print("=== EXAMINING EXISTING INTERFACES ===")

    for device in devices:
        device_id = device['node_id']
        device_type = device['type']
        print(f"\n--- Interfaces on {device_id} ({device_type}) ---")

        interfaces = interface_manager.get_device_interfaces(device_id)

        if interfaces:
            print(f"Total interfaces found: {len(interfaces)}")

            # Show first few interface names
            for i, iface in enumerate(interfaces[:50]):
                name = iface.get('name', 'Unknown')
                iface_type = iface.get('type', 'unknown').split(':')[-1] if ':' in iface.get('type', '') else iface.get('type', 'unknown')
                admin_state = iface.get('administrative-state', 'unknown')
                oper_state = iface.get('operational-state', 'unknown')
                print(f"  {i+1}. {name} ({iface_type})")
                # print(f"     Admin: {admin_state}, Oper: {oper_state}")

        else:
            print("No interfaces found or unable to retrieve")

else:
    print("Cannot examine interfaces - no devices available")

## EXERCISE: Create Transponder Interfaces
Fill in the blanks (___) to complete the code

In [ ]:
if interface_manager and devices:
    print("=== EXERCISE: CREATE TRANSPONDER INTERFACES ===")

    # Find the transponder device
    tpdr_devices = [d for d in devices if d['type'] == 'TRANSPONDER']

    if tpdr_devices:
        device_id = tpdr_devices[0]['node_id']

        print(f"Exercise Device: {device_id}")
        print(f"TransportPCE URL: {interface_manager.base_url}")

        print(f"\n--- Part A: Build the Payload ---")
        print("Fill in the blanks to complete the payload:")

        # EXERCISE: Complete the payload
        # Hint 1: operation should be "create" to create new interfaces
        # Hint 2: Use SERVICE_PARAMS for optical parameters
        # Hint 3: XPDR1-CLIENT1 is the client side, XPDR1-NETWORK1 is network side

        payload = {
            **SERVICE_PARAMS,
            "operation": "___",  # TODO: What operation? (create/delete/modify)
            "service-name": f"student{STUDENT_ID}-tpdr-service",
            "wave-number": "0",
            "nodes": [{
                "node-id": device_id,
                "src-tp": "___",     # TODO: Source termination point (CLIENT side)
                "dest-tp": "___"     # TODO: Destination termination point (NETWORK side)
            }]
        }

        print(f"\nYour payload (with blanks):")
        print(json.dumps(payload, indent=2))

        print(f"\n--- Part B: Complete the API Endpoint ---")

        # EXERCISE: Complete the endpoint
        # Hint: The service is "transportpce-device-renderer:service-path"
        endpoint = "/rests/operations/___:___"  # TODO: Complete the service name

        print(f"Your endpoint: {endpoint}")

        print(f"\n--- Part C: Choose HTTP Method ---")
        # EXERCISE: What HTTP method for executing an operation?
        # Hint: GET=read, POST=create/execute, PUT=update, DELETE=remove
        http_method = "___"  # TODO: GET, POST, PUT, or DELETE?

        print(f"Your method: {http_method}")

        print("\n" + "="*50)
        print("Check your answers above, then see the solution below")
        print("="*50)

    else:
        print("No transponder device found")
else:
    print("Cannot run exercise - missing prerequisites")

## Solution: Complete Working Code

Here's the complete solution with all blanks filled in:

In [ ]:
# SOLUTION: Complete code to create transponder interfaces

if interface_manager and devices:
    print("=== SOLUTION: CREATE TRANSPONDER INTERFACES ===")
    
    # Find the transponder device
    tpdr_devices = [d for d in devices if d['type'] == 'TRANSPONDER']
    
    if tpdr_devices:
        device_id = tpdr_devices[0]['node_id']
        
        print(f"Device: {device_id}")
        
        # Get interfaces BEFORE
        interfaces_before = interface_manager.get_device_interfaces(device_id)
        print(f"Interfaces before: {len(interfaces_before)}")
        
        # Complete payload with correct values
        payload = {
            **SERVICE_PARAMS,
            "operation": "create",              # ANSWER: create
            "service-name": f"student{STUDENT_ID}-tpdr-service",
            "wave-number": "0",
            "nodes": [{
                "node-id": device_id,
                "src-tp": "XPDR1-CLIENT1",     # ANSWER: XPDR1-CLIENT1
                "dest-tp": "XPDR1-NETWORK1"    # ANSWER: XPDR1-NETWORK1
            }]
        }
        
        # Complete endpoint
        endpoint = "/rests/operations/transportpce-device-renderer:service-path"  # ANSWER
        url = f"{interface_manager.base_url}{endpoint}"
        
        # HTTP method
        http_method = "POST"  # ANSWER: POST for operations
        
        print(f"\nMaking API call...")
        print(f"URL: {url}")
        print(f"Method: {http_method}")
        print(f"Payload: {json.dumps(payload, indent=2)}")
        
        # Make the request
        response = interface_manager.session.post(
            url,
            headers=interface_manager.headers,
            json={"input": payload}
        )
        
        print(f"\nResponse: {response.status_code}")
        print(f"Response: {response.text}")
        
        if response.status_code == 200:
            print(f"\n✓ SUCCESS!")
            
            # Wait and check
            time.sleep(5)
            
            # Get interfaces AFTER
            interfaces_after = interface_manager.get_device_interfaces(device_id)
            print(f"\nInterfaces after: {len(interfaces_after)}")
            
            # Show new interfaces
            new_count = len(interfaces_after) - len(interfaces_before)
            if new_count > 0:
                print(f"Created {new_count} new interfaces")
                
                # Show XPDR interfaces
                xpdr_interfaces = [i for i in interfaces_after if 'XPDR' in i.get('name', '')]
                if xpdr_interfaces:
                    print(f"\nXPDR interfaces found:")
                    for iface in xpdr_interfaces:
                        print(f"  - {iface.get('name')}")
        else:
            print(f"\n✗ Failed with status {response.status_code}")
            
    else:
        print("No transponder device found")
else:
    print("Cannot run - missing prerequisites")

## Examine Interfaces After Configuration

Now let's check the interfaces again to see what was created:

In [ ]:
if interface_manager and devices:
    print("=== EXAMINING EXISTING INTERFACES ===")

    for device in devices:
        device_id = device['node_id']
        device_type = device['type']
        print(f"\n--- Interfaces on {device_id} ({device_type}) ---")

        interfaces = interface_manager.get_device_interfaces(device_id)

        if interfaces:
            print(f"Total interfaces found: {len(interfaces)}")

            # Show first few interface names
            for i, iface in enumerate(interfaces[:50]):
                name = iface.get('name', 'Unknown')
                iface_type = iface.get('type', 'unknown').split(':')[-1] if ':' in iface.get('type', '') else iface.get('type', 'unknown')
                admin_state = iface.get('administrative-state', 'unknown')
                oper_state = iface.get('operational-state', 'unknown')
                print(f"  {i+1}. {name} ({iface_type})")
                # print(f"     Admin: {admin_state}, Oper: {oper_state}")

        else:
            print("No interfaces found or unable to retrieve")

else:
    print("Cannot examine interfaces - no devices available")